In [1]:
import pandas as pd
import numpy as np
import pyreadr
from scipy.spatial import distance
from tqdm import tqdm

import shapefile as shp  # Requires the pyshp package
import matplotlib.pyplot as plt

SUBSET = False #make this true if you want to test with subset of the dataframe

In [2]:
def summarize_table(df_in):
    print(f'Rows: {df.shape[0]}, Cols: {df.shape[1]}')

    # Check for non-zero NA values in each column
    na_counts = df_in.isna().sum()
    columns_with_na = na_counts[na_counts > 0].index
    if len(columns_with_na) > 0:
        print("Columns with non-zero NA values:")
        print(columns_with_na)
    else:
        print("No NA values found in any column.")

In [3]:
#read in the main rdata file as dataframe 
analysis_path = '/Users/annieulichney/Desktop/Deforestation/analysis.Rdata'

main = pyreadr.read_r(analysis_path) 

df = pd.DataFrame(main['forest_full'])
if SUBSET: df = df.sample(5000)


summarize_table(df)

Rows: 2427898, Cols: 35
No NA values found in any column.


## Economic Data

In [4]:
#calculate total Produto Interno Bruto/Gross Domestic Product (GDP) 
df['PIB'] = df['pib_pc'] * df['populacao']

## Labor and Social Security (RAIS) Data

In [5]:
rais_path = '/Users/annieulichney/Desktop/Deforestation/rais_setor_munic.Rdata'
rais_read = pyreadr.read_r(rais_path)
rais_df = pd.DataFrame(rais_read['rais3'])
rais_df.head()

,id_municipio,ano,n_empresas_ADMIN PUBLICA,n_empresas_AGRICULTURA,n_empresas_ALIMENTOS E BEBIDAS,n_empresas_ALOJAMENTO E ALIMENTACAO,n_empresas_ALUGUEL EQUIPAMENTO,n_empresas_ATACADO,n_empresas_ATIVIDADES ASSOCIATIVAS,n_empresas_AUTOMOVEIS E TRANSPORTE,...,n_empregos_IMOBILIARIA,n_empregos_RECICLAGEM,n_empregos_COUROS E CALCADO,n_empregos_BORRACHA E PLASTICO,n_empregos_TEXTEIS,n_empregos_PESQUISA E DESENVOLVIMENTO,n_empregos_TRANSPORTE AEREO,n_empregos_FUMO,n_empregos_REFINO PETROLEO,n_empregos_
0,1100015,2000,2,27,15,13,1,38,126,5,...,0,0,0,0,0,0,0,0,0,0
1,1100015,2001,3,32,17,10,1,40,118,4,...,0,0,0,0,0,0,0,0,0,0
2,1100015,2002,3,45,23,13,0,39,127,4,...,0,0,0,0,0,0,0,0,0,0
3,1100015,2003,2,65,23,13,0,33,109,4,...,0,0,0,0,0,0,0,0,0,0
4,1100015,2004,2,83,44,10,1,29,110,3,...,0,0,0,0,0,0,0,0,0,0


In [6]:
rais_df.shape

(111258, 106)

In [7]:
#translate columns to english
translated_columns = ['PUBLIC ADMIN',
                    'AGRICULTURE',
                    'FOOD AND DRINKS',
                    'ACCOMODATION AND FOOD',
                    'EQUIPMENT RENTAL',
                    'WHOLESALE',
                    'ASSOCIATIVE ACTIVITIES',
                    'AUTOMOBILES AND TRANSPORT',
                    'FINANCIAL ASSISTANCE',
                    'TRADE REP VEHICLES',
                    'CONSTRUCTION',
                    'MAIL AND TELECOM',
                    'CULTURE AND SPORT',
                    'EDITING AND PRINTING',
                    'EDUCATION',
                    'ELECTRICITY AND GAS',
                    'FINANCES',
                    'CLEANING AND SEWAGE',
                    'MACHINERY',
                    'BASIC METALLURGY',
                    'MINING',
                    'WOOD PROD',
                    'NON-METALLIC MINERAL PRODUCTS',
                    'HEALTH',
                    'SERVICES FOR COMPANIES',
                    'PERSONAL SERVICES',
                    'TRANSPORTATION',
                    'GROUND TRANSPORT',
                    'WATER TREATMENT AND DISTRIBUTION',
                    'RETAIL',
                    'COMPUTING',
                    'INSURANCE AND SOCIAL SECURITY',
                    'METALLIC PRODUCTS',
                    'DOMESTIC SERVICES',
                    'FORESTRY',
                    'CLOTHING',
                    'PAPER',
                    'INTERNATIONAL BODIES',
                    'OIL AND GAS',
                    'FISHING AND AQUACULTURE',
                    'CHEMICALS',
                    'WATER-BASED TRANSPORTATION',
                    'REAL ESTATE',
                    'RECYCLING',
                    'LEATHERS AND FOOTWEAR',
                    'RUBBER AND PLASTIC',
                    'TEXTILES',
                    'RESEARCH AND DEVELOPMENT',
                    'AERO TRANSPORT',
                    'SMOKE',
                    'PETROLEUM REFINING']

translated_companies = ['n_companies_' + t for t in translated_columns]
translated_jobs = ['n_jobs_' + t for t in translated_columns]

column_names = ['id_municipio', 'ano'] + translated_companies + ['n_companies_'] + translated_jobs + ['n_jobs_']
rais_df.columns = column_names

jobs_columns_list = [col for col in rais_df.columns if col.startswith('n_jobs')]
companies_columns_list = [col for col in rais_df.columns if col.startswith('n_companies')]

rais_df['n_companies_TOTAL'] = rais_df[companies_columns_list].sum(axis=1)
rais_df['n_jobs_TOTAL'] = rais_df[jobs_columns_list].sum(axis=1)


In [8]:
#sort the 51 categories of companies into industrial and service oriented

industrial = ['AGRICULTURE',
 'AUTOMOBILES AND TRANSPORT',
 'TRADE REP VEHICLES',
 'WOOD PROD',
 'NON-METALLIC MINERAL PRODUCTS',
 'BASIC METALLURGY',
 'MINING',
 'TRANSPORTATION',
 'GROUND TRANSPORT',
 'WATER-BASED TRANSPORTATION',
 'AERO TRANSPORT',
 'CONSTRUCTION',
 'OIL AND GAS',
 'MACHINERY',
 'FORESTRY',
 'ELECTRICITY AND GAS',
 'FISHING AND AQUACULTURE',
 'CHEMICALS',
 'METALLIC PRODUCTS',
 'PAPER',
 'WATER TREATMENT AND DISTRIBUTION',
 'CLEANING AND SEWAGE',
 'RECYCLING',
 'RUBBER AND PLASTIC',
 'SMOKE',
 'PETROLEUM REFINING']

industrial_companies = ['n_companies_' + i for i in industrial]
industrial_jobs = ['n_jobs_' + i for i in industrial]

service = ['PUBLIC ADMIN',
 'FOOD AND DRINKS',
 'ACCOMODATION AND FOOD',
 'EQUIPMENT RENTAL',
 'WHOLESALE',
 'LEATHERS AND FOOTWEAR',
 'TEXTILES',
 'ASSOCIATIVE ACTIVITIES',
 'FINANCIAL ASSISTANCE',
 'HEALTH',
 'SERVICES FOR COMPANIES',
 'PERSONAL SERVICES',
 'MAIL AND TELECOM',
 'EDITING AND PRINTING',
 'EDUCATION',
 'DOMESTIC SERVICES',
 'INSURANCE AND SOCIAL SECURITY',
 'RETAIL',
 'FINANCES',
 'CULTURE AND SPORT',
 'REAL ESTATE',
 'RESEARCH AND DEVELOPMENT',
 'COMPUTING',
 'CLOTHING',
 'INTERNATIONAL BODIES']

service_companies = ['n_companies_' + i for i in service]
service_jobs = ['n_jobs_' + i for i in service]


transportation_logistics = ['AUTOMOBILES AND TRANSPORT',
                    'TRADE REP VEHICLES',
                    'TRANSPORTATION',
                    'GROUND TRANSPORT',
                    'WATER-BASED TRANSPORTATION',
                    'AERO TRANSPORT']
                    
transportation_companies = ['n_companies_' + i for i in transportation_logistics]
transportation_jobs = ['n_jobs_' + i for i in transportation_logistics]

construction_manufacturing = ['CONSTRUCTION',
                            'MACHINERY',
                            'BASIC METALLURGY',
                            'MINING',
                            'WOOD PROD',
                            'NON-METALLIC MINERAL PRODUCTS',
                            'METALLIC PRODUCTS', 
                            'CHEMICALS']

construction_manufacturing_companies = ['n_companies_' + i for i in construction_manufacturing]
construction_manufacturing_jobs = ['n_jobs_' + i for i in construction_manufacturing]

oil_gas = [   'ELECTRICITY AND GAS',
              'OIL AND GAS',
              'SMOKE',
              'PETROLEUM REFINING']

oil_gas_companies = ['n_companies_' + i for i in oil_gas]
oil_gas_jobs = ['n_jobs_' + i for i in oil_gas]

forestry = ['WOOD PROD',
              'FORESTRY',
              'PAPER']
              
forestry_companies = ['n_companies_' + i for i in forestry]
forestry_jobs = ['n_jobs_' + i for i in forestry]

public_admin = ['PUBLIC ADMIN']
public_admin_companies = ['n_companies_' + i for i in public_admin]
public_admin_jobs = ['n_jobs_' + i for i in public_admin]

#group 
rais_df['n_jobs_TOTAL INDUSTRIAL'] = rais_df[industrial_jobs].sum(axis=1)
rais_df['n_companies_TOTAL INDUSTRIAL'] = rais_df[industrial_companies].sum(axis=1)

rais_df['n_jobs_TOTAL SERVICE'] = rais_df[service_jobs].sum(axis=1)
rais_df['n_companies_TOTAL SERVICE'] = rais_df[service_companies].sum(axis=1)

rais_df['n_jobs_TOTAL TRANSPORTATION'] = rais_df[transportation_jobs].sum(axis=1)
rais_df['n_companies_TOTAL TRANSPORTATION'] = rais_df[transportation_companies].sum(axis=1)

rais_df['n_jobs_TOTAL CONSTRUCTION AND MANUFACTURING'] = rais_df[construction_manufacturing_jobs].sum(axis=1)
rais_df['n_companies_TOTAL CONSTRUCTION AND MANUFACTURING'] = rais_df[construction_manufacturing_companies].sum(axis=1)

rais_df['n_jobs_TOTAL OIL AND GAS'] = rais_df[oil_gas_jobs].sum(axis=1)
rais_df['n_companies_TOTAL OIL AND GAS'] = rais_df[oil_gas_companies].sum(axis=1)

rais_df['n_jobs_TOTAL FORESTRY'] = rais_df[forestry_jobs].sum(axis=1)
rais_df['n_companies_TOTAL FORESTRY'] = rais_df[forestry_companies].sum(axis=1)

rais_df['n_jobs_TOTAL PUBLIC ADMIN'] = rais_df[public_admin_jobs].sum(axis=1)
rais_df['n_companies_TOTAL PUBLIC ADMIN'] = rais_df[public_admin_companies].sum(axis=1)

In [9]:
#convert to percent

jobs_columns_list_including_new = [col for col in rais_df.columns if (col.startswith('n_jobs') and col!='n_jobs_TOTAL')]
companies_columns_list_including_new = [col for col in rais_df.columns if (col.startswith('n_companies') and col!='n_companies_TOTAL')]

rais_df[jobs_columns_list_including_new] = rais_df[jobs_columns_list_including_new].div(rais_df.n_jobs_TOTAL, axis=0)
rais_df[companies_columns_list_including_new] = rais_df[companies_columns_list_including_new].div(rais_df.n_companies_TOTAL, axis=0)
rais_df.head()

,id_municipio,ano,n_companies_PUBLIC ADMIN,n_companies_AGRICULTURE,n_companies_FOOD AND DRINKS,n_companies_ACCOMODATION AND FOOD,n_companies_EQUIPMENT RENTAL,n_companies_WHOLESALE,n_companies_ASSOCIATIVE ACTIVITIES,n_companies_AUTOMOBILES AND TRANSPORT,...,n_jobs_TOTAL TRANSPORTATION,n_companies_TOTAL TRANSPORTATION,n_jobs_TOTAL CONSTRUCTION AND MANUFACTURING,n_companies_TOTAL CONSTRUCTION AND MANUFACTURING,n_jobs_TOTAL OIL AND GAS,n_companies_TOTAL OIL AND GAS,n_jobs_TOTAL FORESTRY,n_companies_TOTAL FORESTRY,n_jobs_TOTAL PUBLIC ADMIN,n_companies_TOTAL PUBLIC ADMIN
0,1100015,2000,0.003711,0.050093,0.027829,0.024119,0.001855,0.070501,0.233766,0.009276,...,0.069398,0.083488,0.154682,0.066790,0.010870,0.014842,0.130435,0.042672,0.442308,0.003711
1,1100015,2001,0.005525,0.058932,0.031308,0.018416,0.001842,0.073665,0.217311,0.007366,...,0.072574,0.090239,0.151055,0.060773,0.027004,0.014733,0.124895,0.040516,0.431224,0.005525
2,1100015,2002,0.004992,0.074875,0.038270,0.021631,0.000000,0.064892,0.211314,0.006656,...,0.066667,0.084859,0.148611,0.063228,0.026389,0.014975,0.115278,0.034942,0.447917,0.004992
3,1100015,2003,0.003190,0.103668,0.036683,0.020734,0.000000,0.052632,0.173844,0.006380,...,0.056476,0.079745,0.157766,0.057416,0.026397,0.015949,0.106814,0.031898,0.430939,0.003190
4,1100015,2004,0.003125,0.129688,0.068750,0.015625,0.001563,0.045312,0.171875,0.004687,...,0.081988,0.085938,0.123602,0.051562,0.028571,0.009375,0.080124,0.029687,0.361491,0.003125


In [10]:
df = pd.merge(df, rais_df,  how = 'left', left_on = ['ID','year'], right_on = ['id_municipio','ano'])

In [11]:
summarize_table(df)

Rows: 2427898, Cols: 158
No NA values found in any column.


In [12]:
df.head()

,ID,forest.diff,FID,x,y,year,forest.l,nn_forest.l,rain1,elevation,...,n_jobs_TOTAL TRANSPORTATION,n_companies_TOTAL TRANSPORTATION,n_jobs_TOTAL CONSTRUCTION AND MANUFACTURING,n_companies_TOTAL CONSTRUCTION AND MANUFACTURING,n_jobs_TOTAL OIL AND GAS,n_companies_TOTAL OIL AND GAS,n_jobs_TOTAL FORESTRY,n_companies_TOTAL FORESTRY,n_jobs_TOTAL PUBLIC ADMIN,n_companies_TOTAL PUBLIC ADMIN
0,1100015,0.0,8436645,-62.825,-12.925,2002,97.0,85.50,21.81664,154.0,...,0.066667,0.084859,0.148611,0.063228,0.026389,0.014975,0.115278,0.034942,0.447917,0.004992
1,1100015,16.0,8436646,-62.825,-12.875,2002,65.0,68.75,22.56391,146.0,...,0.066667,0.084859,0.148611,0.063228,0.026389,0.014975,0.115278,0.034942,0.447917,0.004992
2,1100015,5.0,8440244,-62.775,-12.975,2002,92.0,87.75,21.30135,156.0,...,0.066667,0.084859,0.148611,0.063228,0.026389,0.014975,0.115278,0.034942,0.447917,0.004992
3,1100015,9.0,8440245,-62.775,-12.925,2002,83.0,73.75,22.10585,156.0,...,0.066667,0.084859,0.148611,0.063228,0.026389,0.014975,0.115278,0.034942,0.447917,0.004992
4,1100015,13.0,8443844,-62.725,-12.975,2002,79.0,80.25,20.90614,155.0,...,0.066667,0.084859,0.148611,0.063228,0.026389,0.014975,0.115278,0.034942,0.447917,0.004992


## Year Deforest Data Set

In [13]:
#Codo and Mojui Dos Campos are two municipalities where year_deforest introduces many na's:


#First we removes codo the municipality that is not in the bounds of legal amazon
df = df[df.ID != 2103307]

# Mojui Dos Campos is a newly-created municipality, liberated from Santarem in 2006
#fill in the missing points with the munipcality that is closest to it before the name change

mojui_id = 1504752
prior_to_change = df[df.year < 2013]

mojui = df[df.ID == mojui_id]
print(np.unique(mojui.year))
inds_list = mojui.index

[2013 2014 2015 2016]


In [14]:
for i in tqdm(inds_list):
    this_x, this_y, this_year = mojui.x[i], mojui.y[i], mojui.year[i]

    # Calculate distances between the current point and all points in prior_to_change
    distances = np.sqrt((prior_to_change.x - this_x)**2 + (prior_to_change.y - this_y)**2)

    # Find the index of the closest point
    closest_point = np.argmin(distances)

    # Update the 'ID' column in df
    df.at[i, 'ID'] = prior_to_change.at[closest_point, 'ID']

#merge in municipality info
muni_codes = pd.read_csv('/Users/annieulichney/Downloads/Raw_Brazil_GeographicalUnits.csv', encoding= 'ISO-8859-1')
muni_codes = muni_codes[['uf', 'uf_name', 'munic_stub', 'munic_code','munic_name']]
df = pd.merge(df, muni_codes,  how = 'left', left_on = ['ID'], right_on = ['munic_code'])

100%|██████████| 672/672 [00:37<00:00, 17.77it/s]


In [15]:
#merge year deforest data
year_deforest = pyreadr.read_r('/Users/annieulichney/Desktop/Deforestation/year_deforest.RData')
ydf = pd.DataFrame(year_deforest['year_deforest'])
ydf = ydf.drop(['term', 'inc', 'area', 'per', 'assassination', 'assassination_attempt', 'f_emitted_count'], axis = 1)
ydf = ydf.drop_duplicates()
ydf['audits'] = ydf['audits'].fillna(0) #audits that are NA are actually 0

df = pd.merge(df, ydf,  how = 'left', left_on = ['ID','year'], right_on = ['munic_id','year'])

df['audits'] = df['audits'].fillna(0) #fill nas with 0 for audits

In [17]:
#Now calculate emossions variables  'emiss_agr', 'emiss_agropec', 'emiss_pec',
#In the data, variable “atividade_economica” contains the values AGR, PEC and AGROPEC. 
#So, for each municipality/year, we sum the emissions (emissao) that correspond to atividade_economica == “AGROPEC”, “AGR” and “PEC” 

# file_path = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/Rdata/emissions2016.Rdata'
file_path = '/Users/annieulichney/Desktop/Deforestation/emissions2016.Rdata'

emission2016 = pyreadr.read_r(file_path)
emdf = pd.DataFrame(emission2016['df.emissions'])


#agro + pec
both = emdf.loc[emdf['atividade_economica'].isin(['PEC', 'AGR'])][['ano', 'id_municipio', 'atividade_economica', 'emissao']]
both.columns = ['ano', 'id_municipio', 'atividade_economica', 'emissao_agropec_2016']
both['emissao_agropec_2016'] = both['emissao_agropec_2016'].fillna(0)
both_summed = pd.DataFrame(both.groupby(['ano', 'id_municipio'])['emissao_agropec_2016'].sum()).reset_index()
both_summed['id_municipio'] = both_summed['id_municipio'].astype(int)
both_summed['ano'] = both_summed['ano'].astype(int)


#agro only
agro_df = emdf.loc[emdf['atividade_economica'].isin(['AGR'])][['ano', 'id_municipio', 'atividade_economica', 'emissao']]
agro_df.columns = ['ano', 'id_municipio', 'atividade_economica', 'emissao_agr_2016']
agro_df['emissao_agr_2016'] = agro_df['emissao_agr_2016'].fillna(0)
agro_df_summed = pd.DataFrame(agro_df.groupby(['ano', 'id_municipio'])['emissao_agr_2016'].sum()).reset_index()
agro_df_summed['id_municipio'] = agro_df_summed['id_municipio'].astype(int)
agro_df_summed['ano'] = agro_df_summed['ano'].astype(int)


#agro only
pec_df = emdf.loc[emdf['atividade_economica'].isin(['PEC'])][['ano', 'id_municipio', 'atividade_economica', 'emissao']]
pec_df.columns = ['ano', 'id_municipio', 'atividade_economica', 'emissao_pec_2016']
pec_df['emissao_pec_2016'] = pec_df['emissao_pec_2016'].fillna(0)
pec_df_summed = pd.DataFrame(pec_df.groupby(['ano', 'id_municipio'])['emissao_pec_2016'].sum()).reset_index()
pec_df_summed['id_municipio'] = pec_df_summed['id_municipio'].astype(int)
pec_df_summed['ano'] = pec_df_summed['ano'].astype(int)

#merge twice to get all 3 columns
emiss_df = pd.merge(agro_df_summed, both_summed,  how = 'left', left_on = ['id_municipio','ano'], right_on = ['id_municipio','ano'])
emiss_df = pd.merge(pec_df_summed, emiss_df,  how = 'left', left_on = ['id_municipio','ano'], right_on = ['id_municipio','ano'])


df = pd.merge(df, emiss_df,  how = 'left', left_on = ['ID','year'], right_on = ['id_municipio','ano'])


df['emissao_pec_2016'] = df['emissao_pec_2016'].fillna(0)
df['emissao_agr_2016'] = df['emissao_agr_2016'].fillna(0)
df['emissao_agropec_2016'] = df['emissao_agropec_2016'].fillna(0)
df['emiss_pec'] = df['emiss_pec'].fillna(0)
df['emiss_agr'] = df['emiss_agr'].fillna(0)
df['emiss_agropec'] = df['emiss_agropec'].fillna(0)
df['emiss_pec_full'] = df['emiss_pec'] + df['emissao_pec_2016']
df['emiss_agr_full'] = df['emiss_agr'] + df['emissao_agr_2016']
df['emiss_agropec_full'] = df['emiss_agropec'] + df['emissao_agropec_2016']



In [18]:
#The remaining na's in crime are few points that are on the border of municipalities. we drop them: 
inds_to_drop = df[(df.murder_threats.isna() & (df.ID.isin([2102374, 2109304, 2111748, 2112704, 2211209, 2911105, 2928901, 5204904])))].index
df = df.drop(inds_to_drop, axis = 0)
df = df.drop(['ano_y', 'id_municipio_y'], axis = 1)

In [20]:
summarize_table(df)


Rows: 2453529, Cols: 188
No NA values found in any column.


## Crime Variables

In [25]:
#missing many entries from 2002, 2003, 2004 crime data, so I don't merge this in. 

crime = pd.read_csv('bfp_crime_landconflicts_data.csv', index_col=0)
crime['sid'] = crime['sid'].astype(np.int32)
crime['year_parsed'] = crime.year.apply(lambda x: x.split('-')[0])
crime.year_parsed = crime.year_parsed.astype(int)

file_path = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/diretorio.Rdata'
diretorio = pyreadr.read_r(file_path)
dirdf = pd.DataFrame(diretorio['df.diretorio'])

crime = pd.merge(crime, dirdf[['id_municipio', 'id_municipio_6']],  how = 'left', left_on = ['sid'], right_on = ['id_municipio_6'])

crime = crime[['sid', 'id_municipio', 'id_municipio_6', 'year_parsed', 'agg_conflict',
                'agg_attemt', 'agg_murder', 'agg_deaththreat', 'firearm_homicide_total',
                'firearm_homicide_total_male', 'firearm_homicide_total_female',
                'homicide_total', 'homicide_total_male', 'homicide_total_female',
                'homicide_total_branca', 'homicide_total_preta',
                'homicide_total_amarela', 'homicide_total_parda',
                'homicide_total_indigena', 'homicide_mean_age']]

crime = crime.fillna(0)

df_m = pd.merge(df, crime,  how = 'left', left_on = ['ID','year'], right_on = ['id_municipio','year_parsed'])

df_m[df_m.homicide_total_branca.isna()].groupby(['year']).size()

## Political Variables

In [ ]:
vote_full = pd.read_csv('PoliticalDataAnalysis.csv', index_col = 0)

In [ ]:
vote_full.head()

,ano,id_municipio,incumbant,term_limited_seat,special,overall_winner_complete_college,overall_winner_idade,overall_winner_feminino,overall_winner_agriculture_job,overall_winner_public_service_job,...,runnerup_partido_PMDB_MDB,runnerup_partido_PSDB,runnerup_partido_DEM,runnerup_partido_PL,runnerup_partido_other,winner_votes,winner_votes_proportion,runnerup_votes,runnerup_votes_proportion,totalvotes
0,2004,2102200,1.0,0,0.0,0.0,45.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,6582.0,0.551210,4807.0,11941.0,11941.0
1,2004,3203007,1.0,0,0.0,1.0,50.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,10613.0,0.679449,5007.0,15620.0,15620.0
2,2004,3203106,1.0,0,0.0,0.0,67.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,4176.0,0.597767,2810.0,6986.0,6986.0
3,2004,3203163,1.0,0,0.0,0.0,47.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,5146.0,0.694280,2266.0,7412.0,7412.0
4,2004,5214838,1.0,0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,2638.0,0.437697,1954.0,6027.0,6027.0


In [ ]:
vote_plus_one = vote_full.copy()
vote_plus_one['ano'] = vote_plus_one['ano'] + 1

vote_plus_two = vote_full.copy()
vote_plus_two['ano'] = vote_plus_two['ano'] + 2

vote_plus_three = vote_full.copy()
vote_plus_three['ano'] = vote_plus_three['ano'] + 3

vote_concat = pd.concat([vote_full, vote_plus_one, vote_plus_two, vote_plus_three])

In [ ]:
df_trim = df[(df.year >= 2004) & (df.year <= 2016)]

In [ ]:
df_pol = pd.merge(df_trim, vote_concat, how = 'left', left_on = ['ID', 'year'], right_on = ['id_municipio', 'ano'])

In [ ]:
df_pol['runnerup_votes_proportion'] = df_pol.runnerup_votes / df_pol.totalvotes
df_pol['vote_participation_proportion'] = df_pol.totalvotes/df_pol.populacao

df_pol = df_pol.drop('totalvotes', axis = 1)
df_pol = df_pol.drop('winner_votes', axis = 1)
df_pol = df_pol.drop('runnerup_votes', axis = 1)
df_pol = df_pol.drop('ano', axis = 1)
df_pol = df_pol.drop('id_municipio', axis = 1)

In [ ]:
if 'Unnamed: 0' in np.array(df_pol.columns):
    df_pol = df_pol.drop('Unnamed: 0', axis = 1)

In [ ]:
df_pol.head()

,ID,forest.diff,FID,x,y,year,forest.l,nn_forest.l,rain1,elevation,...,overall_winner_partido_other,runnerup_partido_PT,runnerup_partido_PMDB_MDB,runnerup_partido_PSDB,runnerup_partido_DEM,runnerup_partido_PL,runnerup_partido_other,winner_votes_proportion,runnerup_votes_proportion,vote_participation_proportion
0,1100015.0,8.0,8436645,-62.825,-12.925,2004,84.0,70.75,23.39269,154.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.5231,0.4769,0.464984
1,1100015.0,21.0,8436646,-62.825,-12.875,2004,48.0,62.75,24.21399,146.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.5231,0.4769,0.464984
2,1100015.0,-19.0,8440244,-62.775,-12.975,2004,92.0,83.50,23.46420,156.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.5231,0.4769,0.464984
3,1100015.0,12.0,8440245,-62.775,-12.925,2004,78.0,68.50,23.58431,156.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.5231,0.4769,0.464984
4,1100015.0,0.0,8443844,-62.725,-12.975,2004,77.0,76.50,23.37501,155.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.5231,0.4769,0.464984


In [ ]:
dict(sorted((df_pol.isna().sum()).to_dict().items(), key=lambda item: item[1], reverse = True))

{'runnerup_votes_proportion': 31273,
 'overall_winner_idade': 13214,
 'incumbant': 81,
 'term_limited_seat': 81,
 'special': 81,
 'overall_winner_complete_college': 81,
 'overall_winner_feminino': 81,
 'overall_winner_agriculture_job': 81,
 'overall_winner_public_service_job': 81,
 'overall_winner_health_job': 81,
 'overall_winner_corporate_job': 81,
 'overall_winner_law_job': 81,
 'overall_winner_technical_job': 81,
 'overall_winner_professional_job': 81,
 'overall_winner_mining_job': 81,
 'overall_winner_partido_PT': 81,
 'overall_winner_partido_PMDB_MDB': 81,
 'overall_winner_partido_PSDB': 81,
 'overall_winner_partido_DEM': 81,
 'overall_winner_partido_PL': 81,
 'overall_winner_partido_other': 81,
 'runnerup_partido_PT': 81,
 'runnerup_partido_PMDB_MDB': 81,
 'runnerup_partido_PSDB': 81,
 'runnerup_partido_DEM': 81,
 'runnerup_partido_PL': 81,
 'runnerup_partido_other': 81,
 'winner_votes_proportion': 81,
 'vote_participation_proportion': 81,
 'ID': 0,
 'forest.diff': 0,
 'FID': 0,

In [ ]:
df_pol[df_pol.runnerup_votes_proportion.isna()].groupby(['year']).size()

year
2004     583
2005     695
2006     695
2007     695
2008    6448
2009    6448
2010    6448
2011    6448
2012     545
2013     545
2014     545
2015     545
2016     633
dtype: int64

In [ ]:
df_pol[df_pol.runnerup_votes_proportion.isna()].groupby(['ID']).size()

ID
1100031.0     176
1101401.0     260
1101807.0     128
1400027.0    3584
1500131.0      84
1500701.0     900
1501105.0     556
1501956.0     324
1502707.0     764
1503903.0    1072
1505502.0    2512
1505635.0     432
1505809.0    3304
1507409.0      64
1508407.0     488
1705557.0      24
1708254.0      84
1710904.0      41
1712157.0      72
1715002.0      56
1715259.0     352
1718865.0     224
1720937.0     148
2101103.0      28
2109700.0      81
2112456.0     192
5102686.0     912
5103361.0     372
5103809.0     132
5104500.0      80
5104526.0     336
5106224.0     318
5106257.0     193
5106307.0    6472
5106703.0     100
5107065.0    2376
5107198.0      88
5107792.0     504
5107875.0    1792
5108105.0     552
5108303.0     600
5108857.0     496
dtype: int64

## Mapbiomas

In [ ]:
#Most relevant variables are forest_formation, forest_lag, forest_diff (= forest_formation - forest_lag) and def (=forest_diff if forest_diff<=0). 
map = pd.read_csv('/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/CSV/mapbiomas_points.csv')

In [ ]:
map.sample(n=30).head()[['forest_formation', 'forest_lag', 'def']]

,forest_formation,forest_lag,def
4490214,0.023995,0.023995,0.0
3768632,2.665895,2.665735,0.0
3043871,0.210504,0.205352,0.0
1989220,3.144578,3.144578,0.0
3661703,3.117989,3.117989,0.0


In [ ]:
map[(map.forest_diff.isna())].groupby(['year']).size()

year
1987    163563
dtype: int64

In [ ]:
df_map = pd.merge(df_pol, map, how = 'left', left_on = ['FID', 'year'], right_on = ['fid_1', 'year'])

In [ ]:
df_map.head()

,ID,forest.diff,FID,x,y,year,forest.l,nn_forest.l,rain1,elevation,...,water,soybean,rice,other_crop,coffee,citrus,other_perennial,forest_lag,forest_diff,def
0,1100015.0,8.0,8436645,-62.825,-12.925,2004,84.0,70.75,23.39269,154.0,...,0.241447,0.0,0.0,0.0,0,0,0.0,2.740301,0.003488,0.0
1,1100015.0,21.0,8436646,-62.825,-12.875,2004,48.0,62.75,24.21399,146.0,...,0.000000,0.0,0.0,0.0,0,0,0.0,0.998124,0.014282,0.0
2,1100015.0,-19.0,8440244,-62.775,-12.975,2004,92.0,83.50,23.46420,156.0,...,0.000000,0.0,0.0,0.0,0,0,0.0,2.973196,0.000000,0.0
3,1100015.0,12.0,8440245,-62.775,-12.925,2004,78.0,68.50,23.58431,156.0,...,0.038767,0.0,0.0,0.0,0,0,0.0,2.923839,0.000872,0.0
4,1100015.0,0.0,8443844,-62.725,-12.975,2004,77.0,76.50,23.37501,155.0,...,0.076749,0.0,0.0,0.0,0,0,0.0,2.894867,0.003138,0.0


In [ ]:
df_map[(df_map.forest_diff.isna())].groupby(['year']).size()

Series([], dtype: int64)

In [ ]:
df_map[(df_map.forest_diff.isna())].groupby(['ID']).size()

Series([], dtype: int64)

In [ ]:
df_map = df_map.drop(['fid_1', 'uniqid'], axis = 1)

In [ ]:
np.array(df_map.columns)

array(['ID', 'forest.diff', 'FID', 'x', 'y', 'year', 'forest.l',
       'nn_forest.l', 'rain1', 'elevation', 'slope', 'aspect',
       'near_mines', 'near_roads', 'near_hidrovia', 'indigenous_homol',
       'mun_election_year', 'new_forest_code', 'lula', 'dilma', 'temer',
       'bolsonaro', 'fed_election_year', 'populacao', 'pib_pc', 'ironore',
       'silver', 'copper', 'gold', 'soy_price', 'beef_price', 'ag_jobs',
       'mining_jobs', 'public_jobs', 'construction_jobs', 'PIB',
       'n_companies_PUBLIC ADMIN', 'n_companies_AGRICULTURE',
       'n_companies_FOOD AND DRINKS', 'n_companies_ACCOMODATION AND FOOD',
       'n_companies_EQUIPMENT RENTAL', 'n_companies_WHOLESALE',
       'n_companies_ASSOCIATIVE ACTIVITIES',
       'n_companies_AUTOMOBILES AND TRANSPORT',
       'n_companies_FINANCIAL ASSISTANCE',
       'n_companies_TRADE REP VEHICLES', 'n_companies_CONSTRUCTION',
       'n_companies_MAIL AND TELECOM', 'n_companies_CULTURE AND SPORT',
       'n_companies_EDITING AND PRIN

In [ ]:
dict(sorted((df_map.isna().sum()).to_dict().items(), key=lambda item: item[1]))

{'ID': 0,
 'forest.diff': 0,
 'FID': 0,
 'x': 0,
 'y': 0,
 'year': 0,
 'forest.l': 0,
 'nn_forest.l': 0,
 'rain1': 0,
 'elevation': 0,
 'slope': 0,
 'aspect': 0,
 'near_mines': 0,
 'near_roads': 0,
 'near_hidrovia': 0,
 'indigenous_homol': 0,
 'mun_election_year': 0,
 'new_forest_code': 0,
 'lula': 0,
 'dilma': 0,
 'temer': 0,
 'bolsonaro': 0,
 'fed_election_year': 0,
 'populacao': 0,
 'pib_pc': 0,
 'ironore': 0,
 'silver': 0,
 'copper': 0,
 'gold': 0,
 'soy_price': 0,
 'beef_price': 0,
 'ag_jobs': 0,
 'mining_jobs': 0,
 'public_jobs': 0,
 'construction_jobs': 0,
 'PIB': 0,
 'n_companies_PUBLIC ADMIN': 0,
 'n_companies_AGRICULTURE': 0,
 'n_companies_FOOD AND DRINKS': 0,
 'n_companies_ACCOMODATION AND FOOD': 0,
 'n_companies_EQUIPMENT RENTAL': 0,
 'n_companies_WHOLESALE': 0,
 'n_companies_ASSOCIATIVE ACTIVITIES': 0,
 'n_companies_AUTOMOBILES AND TRANSPORT': 0,
 'n_companies_FINANCIAL ASSISTANCE': 0,
 'n_companies_TRADE REP VEHICLES': 0,
 'n_companies_CONSTRUCTION': 0,
 'n_companies_MAIL

In [ ]:
for year in [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]:
    filename1 = f'/{str(year)}_1.csv'
    filename2 = f'/{str(year)}_2.csv'
    filename3 = f'/{str(year)}_3.csv'
    filename4 = f'/{str(year)}_4.csv'

    # "    filename1 = f'FinalData/FinalData{str(year)}_1.csv'\n",
    # "    filename2 = f'FinalData/FinalData{str(year)}_2.csv'\n",
    # "    filename3 = f'FinalData/FinalData{str(year)}_3.csv'\n",
    # "    filename4 = f'FinalData/FinalData{str(year)}_4.csv'\n",

    this_year_df = df_map[df_map.year == year]

    num_rows = len(this_year_df)
    quarter_rows = num_rows // 4

    this_year_df.iloc[:quarter_rows].to_csv(filename1)
    this_year_df.iloc[quarter_rows:2*quarter_rows].to_csv(filename2)
    this_year_df.iloc[2*quarter_rows:3*quarter_rows].to_csv(filename3)
    this_year_df.iloc[3*quarter_rows:].to_csv(filename4)
